In [1]:
# !pip install sqlalchemy
# !conda install sqlalchemy -y

import sqlalchemy

In [2]:
# verificar a versão
sqlalchemy.__version__

'1.4.22'

# Conectando ao banco de dados

In [3]:
# criar a interface de conexão ao banco de dados
# configurar o echo como True vai mostrar os códigos SQL gerados

#engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True)

engine = sqlalchemy.create_engine('sqlite:///enterprise.db', echo=True)

# Declarando o mapeamento

Na configuração inicial do ORM vamos iniciar descrevendo as tabelas que serão inseridas no banco, que nesse caso serão as nossas próprias classes

In [4]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [5]:
from sqlalchemy import Column, Integer, String

In [6]:
class User(Base):
    
    __tablename__ = 'users' # obrigatório

    id = Column(Integer, primary_key=True) # obrigatório
    name = Column(String(50))
    fullname = Column(String(50))
    age = Column(Integer)

    def __repr__(self):
        return "<User(name='{}', fullname='{}', age={})>".format(
            self.name, self.fullname, self.age)

# Criar a tabela no banco de dados

In [7]:
# criar a tabela no banco de dados
Base.metadata.create_all(engine)

2021-10-13 16:26:45,850 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:26:45,851 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2021-10-13 16:26:45,852 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-13 16:26:45,853 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2021-10-13 16:26:45,853 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-10-13 16:26:45,854 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	fullname VARCHAR(50), 
	age INTEGER, 
	PRIMARY KEY (id)
)


2021-10-13 16:26:45,855 INFO sqlalchemy.engine.Engine [no key 0.00040s] ()
2021-10-13 16:26:45,859 INFO sqlalchemy.engine.Engine COMMIT


# Criar uma instância da classe mapeada

In [8]:
user = User(name='enzo', fullname='Francisco Enzo', age=20)

In [9]:
user.name

'enzo'

In [10]:
user.fullname

'Francisco Enzo'

In [11]:
str(user.id)

'None'

# Criar uma sessão

In [12]:
from sqlalchemy.orm import sessionmaker

# criando uma sessão e atribuindo a engine criada anteriormente
Session = sessionmaker(bind=engine)
Session

sessionmaker(class_='Session', bind=Engine(sqlite:///enterprise.db), autoflush=True, autocommit=False, expire_on_commit=True)

In [13]:
# criar um objeto da sessão
session = Session()

# Adicionar objetos (INSERT)

In [14]:
# persistindo o objeto criado na sessão
session.add(user)

Até agora, dizemos que a instância está pendente; nenhum SQL ainda foi criado e o objeto ainda não foi representado por uma linha no banco de dados. A sessão só enviará o SQL para persistir o usuário assim que for necessário, usando um processo conhecido como **flush**.

In [15]:
# confirmar as mudanças
session.commit()

2021-10-13 16:26:52,107 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:26:52,109 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2021-10-13 16:26:52,110 INFO sqlalchemy.engine.Engine [generated in 0.00063s] ('enzo', 'Francisco Enzo', 20)
2021-10-13 16:26:52,111 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
# criar e adicionar diversos objetos na nossa sessão
session.add_all([
    User(name='valentina', fullname='Maria Valentina', age=23),
    User(name='pedro', fullname='João Pedro', age=20)])

In [17]:
# verificar as novas mudanças pendentes
session.new

IdentitySet([<User(name='valentina', fullname='Maria Valentina', age=23)>, <User(name='pedro', fullname='João Pedro', age=20)>])

In [18]:
# confirmar as mudanças
session.commit()

2021-10-13 16:27:01,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:27:01,513 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2021-10-13 16:27:01,513 INFO sqlalchemy.engine.Engine [cached since 9.404s ago] ('valentina', 'Maria Valentina', 23)
2021-10-13 16:27:01,514 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, age) VALUES (?, ?, ?)
2021-10-13 16:27:01,514 INFO sqlalchemy.engine.Engine [cached since 9.405s ago] ('pedro', 'João Pedro', 20)
2021-10-13 16:27:01,515 INFO sqlalchemy.engine.Engine COMMIT


# Consultar objetos (SELECT)

In [19]:
# consultar por dados
query_user = session.query(User).filter_by(name='enzo').first()

2021-10-13 16:27:02,912 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:27:02,915 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2021-10-13 16:27:02,915 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ('enzo', 1, 0)


In [20]:
query_user.name

'enzo'

In [21]:
query_user.age

20

In [22]:
# selecionar todos os usuários ordenados pelo o id
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2021-10-13 16:27:16,560 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users ORDER BY users.id
2021-10-13 16:27:16,560 INFO sqlalchemy.engine.Engine [generated in 0.00044s] ()
enzo Francisco Enzo
valentina Maria Valentina
pedro João Pedro


In [23]:
# filtrar por um atributo
for name in session.query(User.name, User.age).filter_by(name='enzo'):
    print(name)

2021-10-13 16:27:23,699 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
2021-10-13 16:27:23,700 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('enzo',)
('enzo', 20)


# Modificar objetos (UPDATE)

In [24]:
user.fullname

'Francisco Enzo'

In [25]:
# mudar os dados do nosso objeto já criado (UPDATE)
user.fullname = 'Francisco Enzo Souza'

In [26]:
# verificar o cache de mudanças
session.dirty

IdentitySet([<User(name='enzo', fullname='Francisco Enzo Souza', age=20)>])

In [27]:
# confirmar as mudanças
session.commit()

2021-10-13 16:27:37,615 INFO sqlalchemy.engine.Engine UPDATE users SET fullname=? WHERE users.id = ?
2021-10-13 16:27:37,616 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('Francisco Enzo Souza', 1)
2021-10-13 16:27:37,617 INFO sqlalchemy.engine.Engine COMMIT


# Deletar objetos (DELETE)

In [28]:
user = session.query(User).filter_by(name='valentina').first()

2021-10-13 16:27:48,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:27:48,652 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2021-10-13 16:27:48,653 INFO sqlalchemy.engine.Engine [cached since 45.74s ago] ('valentina', 1, 0)


In [29]:
user

<User(name='valentina', fullname='Maria Valentina', age=23)>

In [30]:
# deletar um usuário
session.delete(user)

In [31]:
# confirmar as mudanças
session.commit()

2021-10-13 16:27:53,664 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2021-10-13 16:27:53,664 INFO sqlalchemy.engine.Engine [generated in 0.00076s] (2,)
2021-10-13 16:27:53,665 INFO sqlalchemy.engine.Engine COMMIT


In [32]:
# consultar por informações refentes ao usuário deletado
session.query(User).filter_by(name='valentina').count()

2021-10-13 16:27:58,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-10-13 16:27:58,122 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.age AS users_age 
FROM users 
WHERE users.name = ?) AS anon_1
2021-10-13 16:27:58,123 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('valentina',)


0

Você pode consultar mais operações usando o SQLAlchemy na documentação da biblioteca: https://docs.sqlalchemy.org/en/14/orm/tutorial.html

Próximas etapa: 
- aprender a construir relações entre objetos;
- adicionar informações em tabelas pré-existentes;
- configurar as operações em cascata.